## Load Model

In [ ]:
from transformers import BitsAndBytesConfig
import torch

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "nvidia/Llama3-ChatQA-1.5-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map={"": 0},
        quantization_config=quant_config
)

In [ ]:
from transformers import pipeline
from langchain import HuggingFacePipeline

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.2,
    return_full_text= False,
    do_sample = True,
    num_return_sequences=1,
    top_k=10,
    eos_token_id=terminators,
)

hf = HuggingFacePipeline(pipeline=pipe)

## Build RAG Application

### Load From Wikipedia

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

docs = WikipediaLoader(query="Candi Borobudur",lang='id',load_max_docs=2).load()

allDocs = "\n\n".join(doc.page_content for doc in docs)

### Split Text

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)
all_splits = text_splitter.split_text(allDocs)
len(all_splits)

### Vector Store

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="firqaaa/indo-sentence-bert-base")
vectorstore = FAISS.from_texts(all_splits, embedding=embeddings)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

### RetrievalQA

In [ ]:
from langchain.chains import RetrievalQA

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import PromptTemplate, LLMChain

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

ret ="""System: Anda adalah chatbot interaktif yang asik untuk menjawab pertanyaan. Kamu bisa mengambil potongan konteks yang diambil berikut ini untuk menjawab pertanyaan tidak apa untuk bilang tidak tahu. Buatlah jawaban yang ringkas 2 kalimat.

          {context}

          User: {question}

          Assistant:
      """

prompt_retqa = PromptTemplate.from_template(ret)
faiss_sim = RetrievalQA.from_chain_type(
    llm=hf, chain_type='stuff', return_source_documents=True, chain_type_kwargs={"prompt": prompt_retqa},
    retriever=retriever
)

res_sim = faiss_sim('Mengapa Candi Borobudur ditinggalkan?')
res_sim['result']

### Memory

In [ ]:
from langchain.chains import ConversationalRetrievalChain, StuffDocumentsChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ChatMessageHistory, ConversationEntityMemory, ConversationBufferMemory, ConversationSummaryMemory

history = ChatMessageHistory()
history.add_ai_message("Hai!")

memory = ConversationBufferWindowMemory(k=2,
                                        input_key="question",
                                        memory_key="chat_history",
                                        chat_memory=history,
                                        ai_prefix="Assistant",
                                        output_key = "generated_question",
                                        human_prefix="User")

In [ ]:
ret ="""System: Anda adalah chatbot interaktif yang asik untuk menjawab pertanyaan. Kamu bisa mengambil potongan konteks yang diambil berikut ini untuk menjawab pertanyaan tidak apa untuk bilang tidak tahu. Buatlah jawaban yang ringkas 2 kalimat.

          {context}
          {chat_history}
          User: {question}

          Assistant:
"""
prompt_context = PromptTemplate(input_variables=["context", "chat_history", "question"], template=ret)

condense_template ="""SYSTEM: Gabungkan riwayat obrolan dan pertanyaan lanjutan menjadi pertanyaan mandiri.
CHAT_HISTORY : {chat_history}
User: {question}
Assistant:
"""

condense_prompt = PromptTemplate(input_variables=["chat_history", "question"], template=condense_template)

In [ ]:
chain = ConversationalRetrievalChain.from_llm(hf,
                                              retriever=retriever,
                                              memory=memory,
                                              condense_question_prompt=condense_prompt,
                                              return_generated_question= True,
                                              combine_docs_chain_kwargs={'prompt': prompt_context},
                                              get_chat_history=lambda h : h, # fix support with memory
                                              )

In [ ]:
res =  chain("Dimana borobudur terletak")

In [ ]:
res

In [ ]:
answer_text = res['generated_question'].strip()
print(answer_text)
# res['answer'].strip()
# res["generated_question"].strip()

In [12]:
from transformers import pipeline

# Load the pre-trained model and tokenizer
pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"
nlp = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    tokenizer=pretrained_name
)

# # Get input from the user
# user_input = input("Please enter a sentence for sentiment analysis: ")
answer_text = "Memeksaurus"

# Perform sentiment analysis
result = nlp(answer_text)

# Print the result
print("Sentiment analysis result:", result)

Sentiment analysis result: [{'label': 'positive', 'score': 0.9756755828857422}]


In [3]:
def score_sentiment(result):
    return "Positive" if result > 0.75 else "negative"

In [ ]:
score_sentiment(result[0]['score'])